In [102]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os                       # operating system tools (check files)

import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Point, Polygon # also needed

##########################
# Then this stuff below allows us to make a nice inset


from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

ModuleNotFoundError: No module named 'geopandas'

### Create a Map and...

Generate trade exposure metrics for which we can project outcomes at the county/cmz level on changes in trade exposure and/or tariff exposure. The key resource that I am exploiting is the [Quarterly Census of Employment and Wages](https://www.bls.gov/cew/datatoc.htm). 

The process will work like this:

- For 2017, read in the data and construct national level employment and wages by NAICS. Issue here is that we need to be consistent about how stuff is masked at different levels of aggregation. So I will start at county level then work up.


- Merge the national level outcomes with the trade data (at the NAICS) level.


- Then we construct county level "weights". So something like a county's share of national employment by NAICS code, multiply this by total exports of that NAICS code, then sum across all NAICS codes, all for a county. This will aportion exports to a county based on their national level employment share. The sum is a summary measure of a county's exports.


- Outstanding issue is that employment is bottom coded, so small isolated establishments within an NAICS code are not reported. This is where the Dorn strategy of "filling in" might be worth using. For now we will ignore it. 

---

#### Step 1: Read in and clean up the BLS, single file dataset.

In [2]:
url = "https://data.bls.gov/cew/data/files/2017/csv/2017_annual_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

In [3]:
# convert bytes to zip file  
bls_sf = zf.ZipFile(io.BytesIO(r.content)) 
print('Type of zipfile object:', type(bls_sf))

Type of zipfile object: <class 'zipfile.ZipFile'>


In [4]:
bls_sf.namelist()

['2017.annual.singlefile.csv']

In [5]:
clist = ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'disclosure_code', 'annual_avg_estabs',
       'annual_avg_emplvl', 'total_annual_wages','avg_annual_pay']

# These are the columns we care about and will grab

[https://data.bls.gov/cew/doc/titles/area/area_titles.htm](https://data.bls.gov/cew/doc/titles/area/area_titles.htm)

In [6]:
df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)

C:\Users\mwaugh.NYC-STERN\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay
0,1000,0,10,50,0,2017,NaN,124881,1936819,89088710816,45997
1,1000,1,10,51,0,2017,NaN,1208,53131,4339038631,81668
2,1000,1,102,52,0,2017,NaN,1208,53131,4339038631,81668
3,1000,1,1021,53,0,2017,NaN,610,11173,716001109,64083
4,1000,1,1022,53,0,2017,NaN,2,12,369309,30354


In [8]:
df.columns

Index(['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'disclosure_code', 'annual_avg_estabs', 'annual_avg_emplvl',
       'total_annual_wages', 'avg_annual_pay'],
      dtype='object')

#### Step 2: Create National Aggregates

Now what we want to do is to create a national dataset for which we can merge on the county....

In [10]:
NAICS_county_level = 76 
# This is the code that will select only counties at the 4 digit NAICS level
#https://data.bls.gov/cew/doc/titles/agglevel/agglevel_titles.htm

df_county = df[df.agglvl_code == 76].copy()

df_county = df_county[df_county.own_code == 5]
# Only grab private stuff

df_county = df_county[(df_county.area_fips.str[0:2] != "72") & (df_county.area_fips.str[0:2] != "78")
              & (df_county.area_fips.str[0:2] != "02") & (df_county.area_fips.str[0:2] != "15")]
#Drop puerto rico, alaska, hawaii...

df_county["sup_ind"] = df_county.industry_code.str[1].astype(int)
# sometimes there are super industries floating around we want to drop them.
# not clear if this matters with the conditioning all ready

df_county = df_county[df_county["sup_ind"] > 0]

Then once we have this, we group by NAICS code, then sum across employment. This should give, conditional on the county/naics aggregation, the consistent national level totals.

In [11]:
df_national = df_county.groupby("industry_code").agg({"annual_avg_emplvl": "sum"})

In [12]:
df_national.reset_index(inplace = True)

In [13]:
df_national.shape

(304, 2)

---

#### Step 3. Merge national aggregates with trade data

In [14]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

end_use = "naics?get=NAICS,CTY_CODE,ALL_VAL_MO,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use + my_key + "&time==from+2017-01" + "&COMM_LVL=NA4"

url = url + "&CTY_CODE=5700"

In [15]:
r = requests.get(url) 

r

<Response [200]>

In [16]:
dftrade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

dftrade.columns = r.json()[0]

dftrade.time = pd.to_datetime(dftrade.time, format="%Y-%m")
# This is so I can call this correctly...

dftrade.ALL_VAL_MO = dftrade.ALL_VAL_MO.astype(float)

dftrade.head(10)

,NAICS,CTY_CODE,ALL_VAL_MO,CTY_NAME,time,COMM_LVL,CTY_CODE
0,1111,5700,1.931313e+09,CHINA,2017-01-01,NA4,5700
1,1112,5700,1.071322e+06,CHINA,2017-01-01,NA4,5700
2,1113,5700,1.480522e+07,CHINA,2017-01-01,NA4,5700
3,1114,5700,1.512023e+06,CHINA,2017-01-01,NA4,5700
4,1119,5700,1.673608e+08,CHINA,2017-01-01,NA4,5700
5,1121,5700,1.172263e+06,CHINA,2017-01-01,NA4,5700
6,1122,5700,2.740000e+03,CHINA,2017-01-01,NA4,5700
7,1124,5700,3.917830e+05,CHINA,2017-01-01,NA4,5700
8,1125,5700,2.599500e+04,CHINA,2017-01-01,NA4,5700
9,1129,5700,1.046655e+06,CHINA,2017-01-01,NA4,5700


In [17]:
dftrade.set_index("time", inplace = True)

In [19]:
df17naics_trade = dftrade.loc["2017"].groupby("NAICS").agg({"ALL_VAL_MO":"sum"})


Alot going on here, grab 2017, groupby NAICS code, then compute the sum. So for a given NAICS code, this will be summing across all observations, which in this case is across months. Thus this is annual exports, by NAICS codes.

In [20]:
df17naics_trade.head()

,ALL_VAL_MO
NAICS,
1111,1.362627e+10
1112,4.750172e+07
1113,4.519789e+08
1114,1.680611e+07
1119,1.665841e+09


In [21]:
total_trade = df17naics_trade.ALL_VAL_MO.sum()

Then merge it with the national level NAICS. Note the groupby operation above leaves the index as the naics code left is on the industry code. Default here is inner, need to think about if I want to carry around zeros.

In [22]:
df_nation_naics = df_national.merge(df17naics_trade, how = "outer", left_on = "industry_code", right_index = True)


In [24]:
print("number of NAICS codes with trade", df_nation_naics.shape[0])
print("national employment", df_nation_naics.annual_avg_emplvl.sum())
print("Potential China Export Employment",df_nation_naics.annual_avg_emplvl.sum())

number of NAICS codes with trade 307
national employment 107022881.0
Potential China Export Employment 107022881.0


In [25]:
df_nation_naics.head()

,industry_code,annual_avg_emplvl,ALL_VAL_MO
0,1111,37447.0,1.362627e+10
1,1112,78485.0,4.750172e+07
2,1113,180531.0,4.519789e+08
3,1114,121550.0,1.680611e+07
4,1119,47430.0,1.665841e+09


In [26]:
df_nation_naics.rename({"annual_avg_emplvl":"nat_emplvl",
                        "ALL_VAL_MO": "china_exports"}, axis = 1, inplace = True)

In [27]:
df_nation_naics.china_exports.replace(np.nan, 0, inplace = True)

In [28]:
df_nation_naics.head()

,industry_code,nat_emplvl,china_exports
0,1111,37447.0,1.362627e+10
1,1112,78485.0,4.750172e+07
2,1113,180531.0,4.519789e+08
3,1114,121550.0,1.680611e+07
4,1119,47430.0,1.665841e+09


#### Step 4: Clean county level data to line up with national aggregates

What I want to do now is rename and probably drop a bunch of this stuff. Then merge it on the national df on the industry code. A think I need to figure out is to only have the county fips codes

In [29]:
df_county.head(10)

,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay,sup_ind
3017,1001,5,1111,76,0,2017,N,1,0,0,0,1
3020,1001,5,1114,76,0,2017,N,1,0,0,0,1
3023,1001,5,1119,76,0,2017,N,2,0,0,0,1
3027,1001,5,1121,76,0,2017,N,2,0,0,0,1
3030,1001,5,1129,76,0,2017,N,1,0,0,0,1
3034,1001,5,1132,76,0,2017,N,2,0,0,0,1
3037,1001,5,1133,76,0,2017,N,2,0,0,0,1
3041,1001,5,1151,76,0,2017,N,2,0,0,0,1
3045,1001,5,1152,76,0,2017,N,1,0,0,0,1
3048,1001,5,1153,76,0,2017,NaN,6,22,1793086,81196,1


Below this is just exploring some issues with this. One thing that jumps out is not all NAICS codes are represented within a County. At somepoint need to explore why

In [30]:
df_county.annual_avg_emplvl.sum()

107022881

In [31]:
df_nation_naics.nat_emplvl.sum()

107022881.0

In [32]:
grp = df_county.groupby("area_fips")

In [34]:
grp.get_group(10003).head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,avg_annual_pay,sup_ind
332500,10003,5,1111,76,0,2017,N,6,0,0,0,1
332511,10003,5,1112,76,0,2017,N,1,0,0,0,1
332514,10003,5,1114,76,0,2017,NaN,11,125,4337571,34677,1
332522,10003,5,1121,76,0,2017,N,1,0,0,0,1
332525,10003,5,1123,76,0,2017,N,1,0,0,0,1


In [89]:
def create_trade_exposure(df):

    new_df = df.merge(df_nation_naics, how = "outer", left_on = "industry_code", right_on = "industry_code")
    # Merge the nation with the county, why, we want to make sure all the naics codes are lined up properly
    
    naics_codes = df_nation_naics.shape[0]
    
    county_share = (new_df.annual_avg_emplvl/new_df.nat_emplvl)*(new_df.china_exports)
    # Then at the NAICS level, take a county's employment relative th national employment. 
    # This is like the weight. Then multiply it by NAICS exports. So this is like if LA has 5 percernt in X,
    # Then 5 percent of X's exports go to LA. 
        
    trd_exp = (1/new_df.annual_avg_emplvl.sum())*county_share.sum()
    # Then sum acrross all the NAICS codes
    
    foo = {"export_exposure": [trd_exp], "employment": [new_df.annual_avg_emplvl.sum()]}
    
    return pd.DataFrame(foo, index = [new_df.area_fips.iloc[0]])
    

In [90]:
single_location = create_trade_exposure(grp.get_group(10001))

In [91]:
single_location

,export_exposure,employment
10001,1014.35348,20205.0


In [95]:
df_trdx_county = grp.apply(create_trade_exposure)

C:\Users\mwaugh.NYC-STERN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
C:\Users\mwaugh.NYC-STERN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


In [100]:
df_trdx_county = df_trdx_county.droplevel(1)

In [109]:
df_trdx_county.sort_values(by = ["export_exposure"], ascending = False)[100:125]

,export_exposure,employment
area_fips,,
38073,30380.996278,527.0
5017,30333.442083,1250.0
38103,30236.569817,349.0
20027,29973.762615,607.0
17023,29912.086976,627.0
29033,29682.174954,662.0
31095,29425.820194,532.0
31121,29404.563890,693.0
19095,29273.457565,3557.0


In [115]:
df_trdx_county.median()

export_exposure     477.836105
employment         2997.000000
dtype: float64

### Step 5: Make the MAP

In [101]:
cwd = os.getcwd()

regions_shape = cwd + "\\shape_files\\UScounties\\cb_2017_us_county_500k.shx"